<a target="_blank" href="https://colab.research.google.com/github/wbfrench1/barker_DATA606/blob/main/src/spaCy_models/spaCy_NER_model_final.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

Followed this tutorial
https://colab.research.google.com/github/practical-nlp/practical-nlp/blob/master/Ch5/04_NER_using_spaCy%20-%20CoNLL.ipynb#scrollTo=X4wBa1MGwTwy

In [1]:
import os
import pandas as pd
pd.set_option('display.max_rows', 300)
pd.set_option('display.max_columns', 100)
pd.set_option('display.width', 150)
import numpy as np
from sklearn.model_selection import train_test_split
import spacy
from spacy.tokens import DocBin
import pandas as pd
from tqdm import tqdm
import spacy
from spacy.tokens import DocBin

In [2]:
!wget https://github.com/wbfrench1/barker_DATA606/blob/25e862741a599994fdeff7d10de6f6e23e2401a8/src/spaCy_models/config.cfg?raw=True
os.rename('config.cfg?raw=True', 'config.cfg')

--2023-04-23 14:05:18--  https://github.com/wbfrench1/barker_DATA606/blob/25e862741a599994fdeff7d10de6f6e23e2401a8/src/spaCy_models/config.cfg?raw=True
Resolving github.com (github.com)... 140.82.114.3
Connecting to github.com (github.com)|140.82.114.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/wbfrench1/barker_DATA606/raw/25e862741a599994fdeff7d10de6f6e23e2401a8/src/spaCy_models/config.cfg [following]
--2023-04-23 14:05:18--  https://github.com/wbfrench1/barker_DATA606/raw/25e862741a599994fdeff7d10de6f6e23e2401a8/src/spaCy_models/config.cfg
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/wbfrench1/barker_DATA606/25e862741a599994fdeff7d10de6f6e23e2401a8/src/spaCy_models/config.cfg [following]
--2023-04-23 14:05:18--  https://raw.githubusercontent.com/wbfrench1/barker_DATA606/25e862741a599994fdeff7d10de6f6e23e2401a8/src/spaCy_models/config.c

#### Add periods to the end of each sentence

<p>In the test data at https://github.com/explosion/spaCy/blob/master/extra/example_data/ner_example_data/ner-token-per-line-with-pos.iob   it appears that each sentence should have a period at the end of it.</p>

## Reconfigure Dataset

In [3]:
df = pd.read_csv('https://raw.githubusercontent.com/wbfrench1/barker_DATA606/main/data/movie_data_word_pos_iob_w_space_sep.iob', sep=' ')
print(df.shape)
df.head()

(322035, 4)


,Sentence #,Word,POS,Tag
0,1,what,WP,o
1,1,movies,NNS,o
2,1,star,VBP,o
3,1,bruce,NN,b-actor
4,1,willis,NN,i-actor


In [4]:
df.loc[df['Sentence #'] == 12986]

,Sentence #,Word,POS,Tag
139912,12986,this,DT,o
139913,12986,1977,CD,b-year
139914,12986,movie,NN,o
139915,12986,directed,VBN,o
139916,12986,by,IN,o
139917,12986,steven,JJ,b-director
139918,12986,spielberg,NN,i-director
139919,12986,features,VBZ,o
139920,12986,a,DT,b-plot
139921,12986,man,NN,i-plot


In [5]:
df2 = df.copy()

In [6]:
df2['Sentence #'].max()

21987

In [7]:
str_sent= ''
int_pos= 0
dict_entities= {'entities': []}

for word, pos, tag in df2.loc[df2['Sentence #'] == 1, ['Word', 'POS', 'Tag']].values:
    str_sent += word + ' '
    print('word:', word, 'int_pos',int_pos) 
    
    if tag != 'o':
        dict_entities['entities'].append((int_pos, int_pos + len(word), word))
    
    int_pos += len(word) + 1
str_sent = str_sent[:-1]
str_sent

(str_sent, dict_entities)

word: what int_pos 0
word: movies int_pos 5
word: star int_pos 12
word: bruce int_pos 17
word: willis int_pos 23


('what movies star bruce willis',
 {'entities': [(17, 22, 'bruce'), (23, 29, 'willis')]})

In [8]:
str_sent

'what movies star bruce willis'

In [9]:
dict_entities

{'entities': [(17, 22, 'bruce'), (23, 29, 'willis')]}

In [10]:
l_data = []

for sent_num in range(1,df2['Sentence #'].max() + 1):
    str_sent= ''
    int_pos= 0
    dict_entities= {'entities': []}

    for word, pos, tag in df2.loc[df2['Sentence #'] == sent_num, ['Word', 'POS', 'Tag']].values:
        word = str(word)
        str_sent += word + ' '
        #print('word:', word, 'int_pos',int_pos) 
        
        if tag != 'o':
            dict_entities['entities'].append((int_pos, int_pos + len(word), tag))
        
        int_pos += len(word) + 1
    str_sent = str_sent[:-1]
    str_sent

    l_data.append((str_sent, dict_entities))

In [11]:
l_data[-1:]

[('zac efron is the title character in this 2010 fantasy romance based on the novel by ben sherwood',
  {'entities': [(0, 3, 'b-actor'),
    (4, 9, 'i-actor'),
    (41, 45, 'b-year'),
    (46, 53, 'b-genre'),
    (54, 61, 'i-genre'),
    (62, 67, 'b-origin'),
    (68, 70, 'i-origin'),
    (71, 74, 'i-origin'),
    (75, 80, 'i-origin'),
    (81, 83, 'i-origin'),
    (84, 87, 'i-origin'),
    (88, 96, 'i-origin')]})]

In [12]:
# 21987 refers to 21,987 sentences.  I need to divide them into groups.
int_num_quest = len(l_data)
int_train_end = int(.6 * int_num_quest)
int_test_end = int(int_train_end + .2 * int_num_quest)

### Divide the list of questions: Train, Test, Val

In [13]:
l_train = l_data[:int_train_end]
l_test = l_data[int_train_end:int_test_end]
l_val = l_data[int_test_end:]

dict_data = {'train' : (l_train, "./train.spacy"),
             'test'  : (l_test, "./test.spacy"),
             'val'   : (l_val, "./val.spacy") }

### Convert Data to spaCy format

In [15]:
!python -m spacy init fill-config config.cfg config.cfg 

2023-04-23 14:07:20.878403: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [16]:


for str_data_set_name in dict_data.keys():
    
    nlp = spacy.blank("en") # load a new spacy model
    db = DocBin() # create a DocBin object

    for text, annot in tqdm(dict_data[str_data_set_name][0]): # data in previous format
        doc = nlp.make_doc(text) # create doc object from text
        ents = []
        for start, end, label in annot["entities"]: # add character indexes
            span = doc.char_span(start, end, label=label, alignment_mode="contract")
            if span is None:
                print("Skipping entity")
            else:
                ents.append(span)
        doc.ents = ents # label the text with the ents
        db.add(doc)

    db.to_disk(dict_data[str_data_set_name][1]) # save the docbin object

100%|██████████| 4398/4398 [00:02<00:00, 1920.27it/s]


In [17]:
!python -m spacy train config.cfg --output ./output --paths.train ./train.spacy --paths.dev ./test.spacy 

2023-04-23 14:07:42.701760: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
✔ Created output directory: output
ℹ Saving to output directory: output
ℹ Using CPU

=========================== Initializing pipeline ===========================
[2023-04-23 14:07:45,602] [INFO] Set up nlp object from config
[2023-04-23 14:07:45,625] [INFO] Pipeline: ['tok2vec', 'ner']
[2023-04-23 14:07:45,633] [INFO] Created vocabulary
[2023-04-23 14:07:45,634] [INFO] Finished initializing nlp object
[2023-04-23 14:07:50,984] [INFO] Initialized pipeline components: ['tok2vec', 'ner']
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ner']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00     75.19    0.00    0.00    0.00    0.00
  0     200        2

### Prediction

In [18]:
int_quest_num = 7
str_question = l_val[int_quest_num][0]
l_val[int_quest_num]

('which film is directed by paul thomas anderson and stars daniel day lewis as a turn of the century oilman',
 {'entities': [(26, 30, 'b-director'),
   (31, 37, 'i-director'),
   (38, 46, 'i-director'),
   (57, 63, 'b-actor'),
   (64, 67, 'i-actor'),
   (68, 73, 'i-actor')]})

In [19]:
nlp1 = spacy.load("output/model-best") #load the best model
doc = nlp1(str_question) # input sample text

spacy.displacy.render(doc, style="ent", jupyter=True) # display in Jupyter

## Configuration 2 - tag, dep features,  MaxoutWindowEncoder.v2 depth = 6

In [24]:
!wget https://github.com/wbfrench1/barker_DATA606/blob/ad36e86f3b3a820c9fcbb35f27b0ce21faf7404c/src/spaCy_models/config2.cfg?raw=True
os.rename('config2.cfg?raw=True', 'config2.cfg')

--2023-04-23 14:35:19--  https://github.com/wbfrench1/barker_DATA606/blob/ad36e86f3b3a820c9fcbb35f27b0ce21faf7404c/src/spaCy_models/config2.cfg?raw=True
Resolving github.com (github.com)... 140.82.112.3
Connecting to github.com (github.com)|140.82.112.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/wbfrench1/barker_DATA606/raw/ad36e86f3b3a820c9fcbb35f27b0ce21faf7404c/src/spaCy_models/config2.cfg [following]
--2023-04-23 14:35:19--  https://github.com/wbfrench1/barker_DATA606/raw/ad36e86f3b3a820c9fcbb35f27b0ce21faf7404c/src/spaCy_models/config2.cfg
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/wbfrench1/barker_DATA606/ad36e86f3b3a820c9fcbb35f27b0ce21faf7404c/src/spaCy_models/config2.cfg [following]
--2023-04-23 14:35:19--  https://raw.githubusercontent.com/wbfrench1/barker_DATA606/ad36e86f3b3a820c9fcbb35f27b0ce21faf7404c/src/spaCy_models/conf

In [39]:
!python -m spacy init fill-config config2.cfg config2.cfg 

2023-04-23 15:19:38.411145: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
⚠ Nothing to auto-fill: base config is already complete
✔ Saved config
config2.cfg
You can now add your data and train your pipeline:
python -m spacy train config2.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [40]:


for str_data_set_name in dict_data.keys():
    
    nlp = spacy.blank("en") # load a new spacy model
    db = DocBin() # create a DocBin object

    for text, annot in tqdm(dict_data[str_data_set_name][0]): # data in previous format
        doc = nlp.make_doc(text) # create doc object from text
        ents = []
        for start, end, label in annot["entities"]: # add character indexes
            span = doc.char_span(start, end, label=label, alignment_mode="contract")
            if span is None:
                print("Skipping entity")
            else:
                ents.append(span)
        doc.ents = ents # label the text with the ents
        db.add(doc)

    db.to_disk(dict_data[str_data_set_name][1]) # save the docbin object

100%|██████████| 4398/4398 [00:02<00:00, 1986.73it/s]


In [41]:
!python -m spacy train config2.cfg --output ./output --paths.train ./train.spacy --paths.dev ./test.spacy 

2023-04-23 15:19:59.143373: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
ℹ Saving to output directory: output
ℹ Using CPU

=========================== Initializing pipeline ===========================
[2023-04-23 15:20:02,415] [INFO] Set up nlp object from config
[2023-04-23 15:20:02,438] [INFO] Pipeline: ['tok2vec', 'ner']
[2023-04-23 15:20:02,447] [INFO] Created vocabulary
[2023-04-23 15:20:02,449] [INFO] Finished initializing nlp object
[2023-04-23 15:20:07,947] [INFO] Initialized pipeline components: ['tok2vec', 'ner']
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ner']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00     75.19    0.00    0.00    0.00    0.00
  0     200        190.33   8980.78   31.84   47.64   2

## Config 3 - MaxoutWindowEncoder.v2 depth = 4

In [30]:
!wget https://github.com/wbfrench1/barker_DATA606/blob/8f382f79094dc580eafc1f4c28c8d7c5191b46de/src/spaCy_models/config3.cfg?raw=True
os.rename('config3.cfg?raw=True', 'config3.cfg')

--2023-04-23 14:58:29--  https://github.com/wbfrench1/barker_DATA606/blob/8f382f79094dc580eafc1f4c28c8d7c5191b46de/src/spaCy_models/config3.cfg?raw=True
Resolving github.com (github.com)... 140.82.112.3
Connecting to github.com (github.com)|140.82.112.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/wbfrench1/barker_DATA606/raw/8f382f79094dc580eafc1f4c28c8d7c5191b46de/src/spaCy_models/config3.cfg [following]
--2023-04-23 14:58:30--  https://github.com/wbfrench1/barker_DATA606/raw/8f382f79094dc580eafc1f4c28c8d7c5191b46de/src/spaCy_models/config3.cfg
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/wbfrench1/barker_DATA606/8f382f79094dc580eafc1f4c28c8d7c5191b46de/src/spaCy_models/config3.cfg [following]
--2023-04-23 14:58:30--  https://raw.githubusercontent.com/wbfrench1/barker_DATA606/8f382f79094dc580eafc1f4c28c8d7c5191b46de/src/spaCy_models/conf

In [31]:
!python -m spacy init fill-config config3.cfg config3.cfg 

2023-04-23 14:58:45.265446: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Traceback (most recent call last):
  File "/usr/lib/python3.9/runpy.py", line 197, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/usr/lib/python3.9/runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "/usr/local/lib/python3.9/dist-packages/spacy/__main__.py", line 4, in <module>
    setup_cli()
  File "/usr/local/lib/python3.9/dist-packages/spacy/cli/_util.py", line 74, in setup_cli
    command(prog_name=COMMAND)
  File "/usr/local/lib/python3.9/dist-packages/click/core.py", line 1130, in __call__
    return self.main(*args, **kwargs)
  File "/usr/local/lib/python3.9/dist-packages/typer/core.py", line 778, in main
    return _main(
  File "/usr/local/lib/python3.9/dist-packages/typer/core.py", line 216, in _main
    rv = self.invoke(ctx)
  File "/usr/local/lib/python3.9/dist-packages/click/core.py", line 1657, in inv

In [42]:


for str_data_set_name in dict_data.keys():
    
    nlp = spacy.blank("en") # load a new spacy model
    db = DocBin() # create a DocBin object

    for text, annot in tqdm(dict_data[str_data_set_name][0]): # data in previous format
        doc = nlp.make_doc(text) # create doc object from text
        ents = []
        for start, end, label in annot["entities"]: # add character indexes
            span = doc.char_span(start, end, label=label, alignment_mode="contract")
            if span is None:
                print("Skipping entity")
            else:
                ents.append(span)
        doc.ents = ents # label the text with the ents
        db.add(doc)

    db.to_disk(dict_data[str_data_set_name][1]) # save the docbin object

100%|██████████| 4398/4398 [00:01<00:00, 2401.31it/s]


In [33]:
!python -m spacy train config3.cfg --output ./output --paths.train ./train.spacy --paths.dev ./test.spacy 

2023-04-23 14:59:01.118138: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
ℹ Saving to output directory: output
ℹ Using CPU

=========================== Initializing pipeline ===========================
[2023-04-23 14:59:04,043] [INFO] Set up nlp object from config
[2023-04-23 14:59:04,056] [INFO] Pipeline: ['tok2vec', 'ner']
[2023-04-23 14:59:04,061] [INFO] Created vocabulary
[2023-04-23 14:59:04,062] [INFO] Finished initializing nlp object
[2023-04-23 14:59:09,466] [INFO] Initialized pipeline components: ['tok2vec', 'ner']
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ner']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00     75.19    0.00    0.00    0.00    0.00
  0     200        223.52   9312.67   36.24   45.62   3

## Config 4 - TransitionBasedParser.v2 hidden layers 64 -> 96

In [34]:
!wget https://github.com/wbfrench1/barker_DATA606/blob/52ed1ad5b2534a5d16f7f270b03d52443683387e/src/spaCy_models/config4.cfg?raw=True
os.rename('config4.cfg?raw=True', 'config4.cfg')

--2023-04-23 15:10:30--  https://github.com/wbfrench1/barker_DATA606/blob/52ed1ad5b2534a5d16f7f270b03d52443683387e/src/spaCy_models/config4.cfg?raw=True
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/wbfrench1/barker_DATA606/raw/52ed1ad5b2534a5d16f7f270b03d52443683387e/src/spaCy_models/config4.cfg [following]
--2023-04-23 15:10:30--  https://github.com/wbfrench1/barker_DATA606/raw/52ed1ad5b2534a5d16f7f270b03d52443683387e/src/spaCy_models/config4.cfg
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/wbfrench1/barker_DATA606/52ed1ad5b2534a5d16f7f270b03d52443683387e/src/spaCy_models/config4.cfg [following]
--2023-04-23 15:10:30--  https://raw.githubusercontent.com/wbfrench1/barker_DATA606/52ed1ad5b2534a5d16f7f270b03d52443683387e/src/spaCy_models/conf

In [47]:
!python -m spacy init fill-config config4.cfg config4.cfg 

2023-04-23 17:08:34.913851: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
✔ Auto-filled config with all values
✔ Saved config
config4.cfg
You can now add your data and train your pipeline:
python -m spacy train config4.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [48]:


for str_data_set_name in dict_data.keys():
    
    nlp = spacy.blank("en") # load a new spacy model
    db = DocBin() # create a DocBin object

    for text, annot in tqdm(dict_data[str_data_set_name][0]): # data in previous format
        doc = nlp.make_doc(text) # create doc object from text
        ents = []
        for start, end, label in annot["entities"]: # add character indexes
            span = doc.char_span(start, end, label=label, alignment_mode="contract")
            if span is None:
                print("Skipping entity")
            else:
                ents.append(span)
        doc.ents = ents # label the text with the ents
        db.add(doc)

    db.to_disk(dict_data[str_data_set_name][1]) # save the docbin object

100%|██████████| 4398/4398 [00:01<00:00, 2829.83it/s]


In [49]:
!python -m spacy train config4.cfg --output ./output --paths.train ./train.spacy --paths.dev ./test.spacy 

2023-04-23 17:09:06.187311: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
ℹ Saving to output directory: output
ℹ Using CPU

=========================== Initializing pipeline ===========================
[2023-04-23 17:09:09,308] [INFO] Set up nlp object from config
[2023-04-23 17:09:09,321] [INFO] Pipeline: ['tok2vec', 'ner']
[2023-04-23 17:09:09,326] [INFO] Created vocabulary
[2023-04-23 17:09:09,327] [INFO] Finished initializing nlp object
[2023-04-23 17:09:13,695] [INFO] Initialized pipeline components: ['tok2vec', 'ner']
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ner']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00     75.19    0.00    0.00    0.00    0.00
  0     200        324.61   8822.46   29.00   39.79   2

In [50]:
int_quest_num = 7
str_question = l_val[int_quest_num][0]
l_val[int_quest_num]

('which film is directed by paul thomas anderson and stars daniel day lewis as a turn of the century oilman',
 {'entities': [(26, 30, 'b-director'),
   (31, 37, 'i-director'),
   (38, 46, 'i-director'),
   (57, 63, 'b-actor'),
   (64, 67, 'i-actor'),
   (68, 73, 'i-actor')]})

In [51]:
nlp1 = spacy.load("output/model-best") #load the best model
doc = nlp1(str_question) # input sample text

spacy.displacy.render(doc, style="ent", jupyter=True) # display in Jupyter